In [1]:
!pip install tensorflow pymupdf

     ---------------------------------------- 3.6/3.6 MB 16.5 MB/s eta 0:00:00
     --------------------------------------- 24.9/24.9 MB 25.2 MB/s eta 0:00:00


In [7]:
import os
import tensorflow as tf
import fitz  # PyMuPDF

def pdf_to_text(pdf_path):
    """
    Extract text from a single PDF file.
    """
    with fitz.open(pdf_path) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text

def _bytes_feature(value):
    """
    Returns a bytes_list from a string / byte.
    """
    if isinstance(value, type(tf.constant(0))):  # if value is tensor
        value = value.numpy()  # get value of tensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def create_tf_example(pdf_text):
    """
    Create a tf.Example message ready to be written to a file.
    """
    # Create a dictionary mapping the feature name to the tf.Example-compatible data type.
    feature = {
        'text': _bytes_feature(pdf_text.encode('utf-8')),
    }

    # Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

def convert_pdfs_to_tfrecords(pdf_directory, tfrecord_file_name):
    """
    Convert all PDF files in the given directory to a single TFRecord file.
    """
    with tf.io.TFRecordWriter(tfrecord_file_name) as writer:
        for filename in os.listdir(pdf_directory):
            if filename.endswith('.pdf'):
                pdf_path = os.path.join(pdf_directory, filename)
                pdf_text = pdf_to_text(pdf_path)
                tf_example = create_tf_example(pdf_text)
                writer.write(tf_example)

pdf_directory = r'E:\Walsh_work\pdf-tfrecord'
tfrecord_file_name = r'E:\Walsh_work\pdf-tfrecord\pdf_texts.tfrecord'
convert_pdfs_to_tfrecords(pdf_directory, tfrecord_file_name)